In [45]:
import pandas as pd
import numpy as np

In [6]:
"""
Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

если источник traffic_source равен yandex или google, то в source_type ставится organic
для источников paid и email из России - ставим ad
для источников paid и email не из России - ставим other
все остальные варианты берем из traffic_source без изменений
"""

log = pd.read_csv('visit_log.csv', sep=';')
def source_type_func(row):
    if row['traffic_source'] == 'yandex' or row['traffic_source'] == 'google':
        x = 'organic'
    elif row['traffic_source'] == 'paid' or row['traffic_source'] == 'email':
        if row['region'] == 'Russia':
            x = 'ad'
        else:
            x = 'other'
    else:
        x = row['traffic_source']
    return x
    

log['source_type'] = log.apply(source_type_func, axis=1)

log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


In [15]:
"""
В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц
с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию:
/, затем 8 цифр, затем дефис. Выполните следующие действия:

Прочитайте содержимое файла с датафрейм
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии
с заданным шаблоном
"""

pages = pd.read_csv('URLs.txt')
pages[pages.url.str.contains('/\d{8}-', regex=True)].head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [46]:
"""
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей,
которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным
значением столбца timestamp для данного значения userId.
"""

ratings = pd.read_csv('ratings.csv')
ratings['timestamp_max'] = ratings['timestamp']
ratings['timestamp_min'] = ratings['timestamp']
ratings.head()

,userId,movieId,rating,timestamp,timestamp_max,timestamp_min
0,1,31,2.5,1260759144,1260759144,1260759144
1,1,1029,3.0,1260759179,1260759179,1260759179
2,1,1061,3.0,1260759182,1260759182,1260759182
3,1,1129,2.0,1260759185,1260759185,1260759185
4,1,1172,4.0,1260759205,1260759205,1260759205


In [38]:

smth = ratings.pivot_table(index = 'userId', values = ['timestamp', 'timestamp_max', 'timestamp_min'],
                    aggfunc = {'timestamp':'count', 'timestamp_max': np.max, 'timestamp_min': np.min },
                    fill_value = 0, margins = True)
smth.head()

,timestamp,timestamp_max,timestamp_min
userId,,,
1,20,1260759205,1260759108
2,76,835356246,835355395
3,51,1298932787,1298861589
4,204,949982274,949778714
5,100,1163375145,1163373044


In [47]:
smth['delta'] = smth['timestamp_max'] - smth['timestamp_min']
smth = smth[smth.timestamp >= 100]
smth.delta.mean()

41784994.34848485

In [50]:
"""
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.
"""

rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
                  )
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
                   )
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
                  )
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
                          )
print(rzd); print(); print(auto); print(); print(air); print(); print(client_base)

   client_id  rzd_revenue
0        111         1093
1        112         2810
2        113        10283
3        114         5774
4        115          981

   client_id  auto_revenue
0        113         57483
1        114            83
2        115           912
3        116          4834
4        117            98

   client_id  air_revenue
0        115           81
1        116            4
2        117           13
3        118          173

   client_id          address
0        111  Комсомольская 4
1        112   Энтузиастов 8а
2        113  Левобережная 1а
3        114          Мира 14
4        115        ЗЖБИиДК 1
5        116    Строителей 18
6        117  Панфиловская 33
7        118     Мастеркова 4


In [67]:
client_base_x = client_base.merge(rzd, how='outer')
client_base_x = client_base_x.merge(auto, how='outer')
client_base_x = client_base_x.merge(air, how='outer')
client_base_x

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


In [68]:
client_base_y = client_base_x.filter(items = ['client_id', 'rzd_revenue', 'auto_revenue', 'air_revenue'])
client_base_y

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0
